## Purpose:

Try to process a simulation input file, keep the binary output file in memory, extract key variables, and then discard the memory.

### Import

In [45]:
import glob
from pyswmm import Simulation
import pandas as pd
import os
import subprocess
from swmmtoolbox import swmmtoolbox

from contextlib import redirect_stdout

import warnings
warnings.filterwarnings("ignore", category=FutureWarning) # Since SWMMToolbox uses a deprecated function, supress the warning for it

In [46]:
input_files = glob.glob('./simulation_files/delete_test/*.inp')
input_files

['./simulation_files/delete_test/010010201001_rb_lid.inp',
 './simulation_files/delete_test/010010201001_rb.inp',
 './simulation_files/delete_test/010010201001_ng.inp',
 './simulation_files/delete_test/010010201001_rg.inp']

In [56]:
for file in input_files:
    file_dir = file[:file.rfind('.')]  
    
    with redirect_stdout(open(os.devnull, 'w')):  # Redirect PySWMM stdout
        sim = Simulation(file)
        sim.execute()    
    
    os.remove(file_dir + '.rpt')  # Remove the report file
    data = swmmtoolbox.extract(file_dir + '.out', *swmmtoolbox.catalog(file_dir + '.out'))
    data.to_pickle(file_dir + '.pkl')    
    
    print(subprocess.check_output(['du', '-h', file_dir+'.out']).decode('UTF-8'))
    print(subprocess.check_output(['du', '-h', file_dir+'.pkl']).decode('UTF-8'))
    os.remove(file_dir + '.out')
    break

29M	./simulation_files/delete_test/010010201001_rb_lid.out

55M	./simulation_files/delete_test/010010201001_rb_lid.pkl



#### At around 10 variables extracted, the pickle file becomes larger than the output file, and it's no longer worth it to delete it.